In [8]:
import polars as pl

import nwec.utility_reporting.arrearages
import nwec.utils.excel
from nwec.constants import CLEAN_UTILITY_DATA, RAW_UTILITY_DATA

YEAR = 2023
NUM_MONTHS = 6
COLS_PER_MONTH = 9
SHEET_SEARCH_STRING = "Past Due Balances"
ARREARAGE_SEARCH_STRING = "number of past-due balances"
KLI_SEARCH_STRING = "known low-income"
spreadsheet = RAW_UTILITY_DATA / str(YEAR) / "cng.xlsx"

In [9]:
sheet_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, SHEET_SEARCH_STRING)
df = pl.read_excel(spreadsheet, sheet_id=sheet_index, has_header=False)
arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(
    df, NUM_MONTHS, COLS_PER_MONTH, ARREARAGE_SEARCH_STRING
)
kli_arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(df, NUM_MONTHS, COLS_PER_MONTH, KLI_SEARCH_STRING)

# Arrearages

In [10]:
date_to_zip_offset = 1
source_date_format = "%Y-%m-%d %H:%M:%S"

arrearages = nwec.utility_reporting.arrearages.combine_arrearage_year_vintage_cols(
    arrearages, NUM_MONTHS, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [11]:
arrearages.sum()

Zip Code,01 2023 30 Days Arrears,01 2023 30 Days Cust,01 2023 60 Days Arrears,01 2023 60 Days Cust,01 2023 90 Days Arrears,01 2023 90 Days Cust,01 2023 Total Arrearages,02 2023 30 Days Arrears,02 2023 30 Days Cust,02 2023 60 Days Arrears,02 2023 60 Days Cust,02 2023 90 Days Arrears,02 2023 90 Days Cust,02 2023 Total Arrearages,03 2023 30 Days Arrears,03 2023 30 Days Cust,03 2023 60 Days Arrears,03 2023 60 Days Cust,03 2023 90 Days Arrears,03 2023 90 Days Cust,03 2023 Total Arrearages,04 2023 30 Days Arrears,04 2023 30 Days Cust,04 2023 60 Days Arrears,04 2023 60 Days Cust,04 2023 90 Days Arrears,04 2023 90 Days Cust,04 2023 Total Arrearages,05 2023 30 Days Arrears,05 2023 30 Days Cust,05 2023 60 Days Arrears,05 2023 60 Days Cust,05 2023 90 Days Arrears,05 2023 90 Days Cust,05 2023 Total Arrearages,06 2023 30 Days Arrears,06 2023 30 Days Cust,06 2023 60 Days Arrears,06 2023 60 Days Cust,06 2023 90 Days Arrears,06 2023 90 Days Cust,06 2023 Total Arrearages
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
7397559,2.2030e6,10287.0,468038.9,2860.0,677420.3,4374.0,3.3485e6,2.3995e6,10773.0,971746.99,4345.0,657843.99,3642.0,4.0291e6,1.3589e6,5786.0,897098.47,3388.0,864093.54,3789.0,3.1201e6,2.3270e6,11028.0,782139.67,2970.0,1.2782e6,4987.0,4.3873e6,1.3189e6,7166.0,1.0496e6,4070.0,1391526.2,4798.0,3.7600e6,541193.93,3881.0,660744.18,2739.0,1.5275e6,4821.0,2.7294e6


# KLI Arrearages

In [12]:
date_to_zip_offset = 1
source_date_format = "%Y-%m-%d %H:%M:%S"

kli_arrearages = nwec.utility_reporting.arrearages.combine_arrearage_year_vintage_cols(
    kli_arrearages, NUM_MONTHS, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [13]:
kli_arrearages.sum()

Zip Code,01 2023 30 Days Arrears,01 2023 30 Days Cust,01 2023 60 Days Arrears,01 2023 60 Days Cust,01 2023 90 Days Arrears,01 2023 90 Days Cust,01 2023 Total Arrearages,02 2023 30 Days Arrears,02 2023 30 Days Cust,02 2023 60 Days Arrears,02 2023 60 Days Cust,02 2023 90 Days Arrears,02 2023 90 Days Cust,02 2023 Total Arrearages,03 2023 30 Days Arrears,03 2023 30 Days Cust,03 2023 60 Days Arrears,03 2023 60 Days Cust,03 2023 90 Days Arrears,03 2023 90 Days Cust,03 2023 Total Arrearages,04 2023 30 Days Arrears,04 2023 30 Days Cust,04 2023 60 Days Arrears,04 2023 60 Days Cust,04 2023 90 Days Arrears,04 2023 90 Days Cust,04 2023 Total Arrearages,05 2023 30 Days Arrears,05 2023 30 Days Cust,05 2023 60 Days Arrears,05 2023 60 Days Cust,05 2023 90 Days Arrears,05 2023 90 Days Cust,05 2023 Total Arrearages,06 2023 30 Days Arrears,06 2023 30 Days Cust,06 2023 60 Days Arrears,06 2023 60 Days Cust,06 2023 90 Days Arrears,06 2023 90 Days Cust,06 2023 Total Arrearages
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4833653,59171.7,272.0,13062.96,100.0,10538.34,92.0,82773.0,72952.59,254.0,36581.08,183.0,12757.06,98.0,122290.73,46886.43,170.0,38246.19,149.0,27911.4,149.0,113044.02,77713.6,274.0,31271.93,129.0,44158.79,178.0,153144.32,43860.27,163.0,41358.92,159.0,50015.09,186.0,135234.28,27945.36,106.0,30900.19,119.0,61709.75,229.0,120555.3


# Save Output

In [14]:
arrearages = nwec.utility_reporting.arrearages.normalize_arrearage_cols(arrearages, NUM_MONTHS)
arrearages = arrearages.with_columns(pl.lit("CNG").alias("Utility"))
arrearages = arrearages.with_columns(pl.lit("Residential").alias("Customer Class"))

kli_arrearages = nwec.utility_reporting.arrearages.normalize_arrearage_cols(kli_arrearages, NUM_MONTHS)
kli_arrearages = kli_arrearages.with_columns(pl.lit("CNG").alias("Utility"))
kli_arrearages = kli_arrearages.with_columns(pl.lit("KLI").alias("Customer Class"))

pl.concat([arrearages, kli_arrearages]).write_ipc(CLEAN_UTILITY_DATA / "cng.arrow")